# Information

The lambda will be zipped and uplodaded. It needs an role and an existing SNS topic. Here you will find instructions to create both if needed.

A subscription will be made to the SNS topic.

The CLI commands for tags are commented out. They are mainly for information purposes. Nevertheless, remember that tagging resources is a best practice.

# Function information

By default, the function name will be the folder name of the .NET project

In [ ]:
$functionName = Split-Path -Path $pwd -Leaf
Write-Host "Function name: $functionName"

$functionNameLower = ($functionName).ToLower()
Write-Host "Function name lower case: $functionNameLower"

$tagKeyValue = "Key=lambda,Value='$functionNameLower'"
Write-Host "Tag key value: $tagKeyValue"

$tagset = "TagSet=[{$tagKeyValue}]"
Write-Host "Tag set: $tagset"

# Prerequisites

## SNS Topic

In [ ]:
$snsTopicArn = "arn:aws:sns:us-west-1:128163920149:LambdaMinimalApiPublishesOnSNS"
Write-Host "SNS Topic Arn: $snsTopicArn"

If it doesn't exist, create it

In [ ]:
$snsTopicName = $functionName
Write-Host "SNS Topic Name: $snsTopicName" 
$createTopicResponse = aws sns create-topic --name $snsTopicName
$snsTopicArn = ($createTopicResponse | ConvertFrom-Json).TopicArn
Write-Host "SNS Topic Arn: $snsTopicArn"
#aws sns tag-resource --resource $snsTopicArn --tags $tagKeyValue

Role

In [ ]:
$roleName = $functionName
Write-Host "Role name: $roleName"

If it doesnt exists, create it

In [ ]:
aws iam create-role --role-name $roleName --assume-role-policy-document file://policy-document.json

# Deploy

Deploy the function

In [ ]:
dotnet lambda deploy-function --function-name $functionName --function-role $roleName
#aws lambda tag-resource --resource $functionArn --tags $tagKeyValue

Allow the lambda to write into CloudWatch

In [ ]:
aws iam attach-role-policy --policy-arn arn:aws:iam::aws:policy/AWSXRayDaemonWriteAccess --role-name $roleName

Enable tracing

In [ ]:
aws iam attach-role-policy --policy-arn arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole --role-name $roleName

$updateFunctionResponse = aws lambda update-function-configuration --function-name $functionName --tracing-config Mode=Active
$functionArn = ($updateFunctionResponse | ConvertFrom-Json).functionArn
Write-Host "Function Arn: $functionArn"

Subscribe lambda to SNS topic

In [ ]:
aws lambda add-permission --function-name $functionName --statement-id triggerfromsns-statement-id --action lambda:invokefunction --principal sns.amazonaws.com --source-arn $snsTopicArn

In [ ]:
aws sns subscribe --topic-arn $snsTopicArn --protocol lambda --notification-endpoint $functionArn

# <p style="text-align: center;">🎉 Deployed!</p>

# Test it

In [ ]:
$payload = '{
    "name": "Joan Comas",
    "address": "Awesome street 7, Nicetown, Amazingland"
}'
Invoke-WebRequest -Method Post -Uri ($apiGateWayUrl + "users") -ContentType "application/json" -Body $payload

---

# Clean up

In [ ]:
aws iam detach-role-policy --role-name $roleName --policy-arn arn:aws:iam::aws:policy/AmazonSNSFullAccess
aws cloudformation delete-stack --stack-name $functionName
aws sns delete-topic --topic-arn $snsTopicArn
aws s3 rb --force $s3BucketUri